## 5310 Matches

In [41]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [42]:
pd.set_option("display.max_columns", 100)

In [43]:
df1 = pd.read_csv('5310_org_rd1match.csv')

In [44]:
df2 = pd.read_excel('5310_unmatched.xls')

In [45]:
df = pd.concat([df1, df2], ignore_index=True, sort=False)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   organization_name           150 non-null    object 
 1   ntd_id_x                    6 non-null      object 
 2   ntd_id_y                    6 non-null      object 
 3   ntd_id                      29 non-null     object 
 4   agency_name                 35 non-null     object 
 5   name                        109 non-null    object 
 6   doing_business_as           7 non-null      object 
 7   reporter_acronym            27 non-null     object 
 8   alias                       0 non-null      float64
 9   organization_type_x         42 non-null     object 
 10  organization_type_y         86 non-null     object 
 11  roles                       5 non-null      object 
 12  reporter_type               35 non-null     object 
 13  city                        35 non-

In [47]:
df.reported_by_ntd_id.info()

<class 'pandas.core.series.Series'>
RangeIndex: 150 entries, 0 to 149
Series name: reported_by_ntd_id
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 1.3 KB


In [48]:
df.sample()

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,alias,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,compare_acronym,compare_names,in_5310,reported_by_ntd_id,Unnamed: 11
37,Napa Valley Transportation Authority,NaN,NaN,90088,Napa Valley Transportation Authority,Napa Valley Transportation Authority,NaN,NVTA,NaN,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Full Reporter,Napa,218.0,428002.0,"Vine Transit,Yountville Trolley,Calistoga Shut...",Metropolitan Transportation Commission,both,False,True,both,NaN,NaN


In [49]:
#getting rid of the compare cols so we can check again. and some other columns
df = df.drop(columns={'compare_acronym','compare_names','reported_by_ntd_id','Unnamed: 11','alias'})

In [86]:
df.name.fillna(df['agency_name'], inplace=True)

In [87]:
compare_col2 = np.where(df["organization_name"] == df["name"], True, False)
df["compare_names"] = compare_col2
df.compare_names.value_counts()

False    84
True     66
Name: compare_names, dtype: int64

In [88]:
df>>filter(_.compare_names==False)>>filter(_.name.notnull())

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
40,Omnitrans,NaN,NaN,90029,Omnitrans,OmniTrans,NaN,NaN,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Full Reporter,San Bernardino,232.0,836004.0,"OmniTrans,OmniRide,OmniTrans sbX,OmniAccess",Southern California Association of Governments,both,both,False,Name Match; NTD
66,Amador Transit,9R02-91000,9R02-91000,9R02-91000,Amador Regional Transit System,Amador Regional Transit System,NaN,ARTS,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Rural Reporter,Jackson,11.0,1003002.0,"Amador Transit,Amador Transit Dial-A-Ride",NaN,both,left_only,False,"Manual Match, NTD"
67,Sunline Transit Agency,90079,90079,90079,SunLine Transit Agency,SunLine Transit Agency,NaN,NaN,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Full Reporter,Thousand Palms,327.0,833008.0,"SunLine Transit,SunLine School Tripper,SunDial...",Southern California Association of Governments,both,left_only,False,"Manual Match, NTD"
68,Butte County Association of Governments Butte...,90208,90208,90208,Butte County Association of Governments,Butte County Association of Governments,NaN,BCAG,"MPO, COG or Other Planning Agency",Joint Powers Agency,"Regional Transportation Planning Agency,Metrop...",Full Reporter,Chico,47.0,304001.0,"B-Line,B-Line Paratransit",Butte County Association of Governments,both,left_only,False,"Manual Match, NTD"
69,Easy Lift Transportation Inc,90243,NaN,90243,Easy Lift Transportation,Easy Lift Transportation,NaN,NaN,Private-Non-Profit Corporation,NaN,NaN,Reduced Reporter,Goleta,100.0,542014.0,"Easy Lift Dial-A-Ride,Non Emergency Accessible...",Santa Barbara County Association of Governments,both,left_only,False,"Manual Match, NTD"
71,San Francisco Municipal Transportation Agency,90015,90015,90015,City and County of San Francisco,City and County of San Francisco,San Francisco Municipal Transportation Agency ...,SFMTA,"City, County or Local Government Unit or Depar...",City/Town,NaN,Full Reporter,San Francisco,282.0,438003.0,"Muni Metro,SF Access Van,Muni Bus,San Francisc...",Metropolitan Transportation Commission,both,left_only,False,"Manual Match, NTD"
72,Asian Community Center of Sacramento Valley In...,NaN,NaN,NaN,NaN,Asian Community Center of Sacramento Valley,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,NaN,400.0,NaN,ACC Rides Transportation Services,NaN,right_only,left_only,False,"Manual Match, Airtable"
73,Avenidas,NaN,NaN,NaN,NaN,Avenidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401.0,NaN,Avenidas Door-to-Door,NaN,right_only,left_only,False,"Manual Match, Airtable"
74,City of Lafayette: Lamorinda Sprit Van Program,NaN,NaN,NaN,NaN,City of Lafayette,NaN,NaN,City/Town,NaN,NaN,NaN,NaN,416.0,NaN,Lamorinda Spirit Van,NaN,right_only,left_only,False,"Manual Match, Airtable"
75,City of Vista,NaN,NaN,NaN,NaN,City of Vista,NaN,NaN,City/Town,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only,left_only,False,"Manual Match, Airtable"


In [89]:
df.ntd_id.fillna(df['ntd_id_x'], inplace=True)
df.ntd_id.fillna(df['ntd_id_y'], inplace=True)

In [90]:
df._merge.value_counts()

right_only    74
both          35
Name: _merge, dtype: int64

In [91]:
df.in_5310.value_counts()

left_only    84
both         66
Name: in_5310, dtype: int64

In [92]:
def recategorize(df):   
    if (df['_merge']=='right_only') and (df['in_5310'] == 'both'):
        return 'Name Match; Airtable'
    elif (df['_merge']=='both') and (df['in_5310'] == 'both'):
        return 'Name Match; NTD'
    elif pd.isnull(df['_merge']) and (df['in_5310'] == 'left_only'):
        return 'No Match'
    elif (df['_merge']=='right_only') and (df['in_5310'] == 'left_only'):
        return 'Manual Match, Airtable'
    elif (df['_merge']=='both') and (df['in_5310'] == 'left_only'):
        return 'Manual Match, NTD'
    else: 
        return "NaN"

In [93]:
df['merge_status'] = df.apply(recategorize, axis = 1)

In [95]:
df.merge_status.value_counts()

No Match                  41
Name Match; Airtable      37
Manual Match, Airtable    37
Name Match; NTD           29
Manual Match, NTD          6
Name: merge_status, dtype: int64

In [101]:
df.sample()

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
127,Howard Prep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,False,No Match


In [97]:
(df>>select(_.organization_type_x, _.organization_type_y)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   organization_type_x  42 non-null     object
 1   organization_type_y  86 non-null     object
dtypes: object(2)
memory usage: 2.5+ KB


In [98]:
df.organization_type_y.fillna(df['organization_type_x'], inplace=True)


In [99]:
df>>filter(_.organization_type_x.notnull())

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
8,City of Rio Vista,NaN,NaN,9R02-91014,City of Rio Vista,City of Rio Vista,Rio Vista Delta Breeze,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Rural Reporter,Rio Vista,264.0,448003.0,Rio Vista Delta Breeze,Metropolitan Transportation Commission,both,both,True,Name Match; NTD
9,City of Roseville,NaN,NaN,90168,City of Roseville,City of Roseville,Roseville Transit,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Reduced Reporter,Roseville,271.0,331005.0,"Roseville Transit,Roseville Dial-A-Ride",Sacramento Area Council of Governments,both,both,True,Name Match; NTD
10,City of Santa Rosa,NaN,NaN,90017,City of Santa Rosa,City of Santa Rosa,NaN,NaN,"City, County or Local Government Unit or Depar...",City/Town,NaN,Full Reporter,Santa Rosa,301.0,449002.0,"Santa Rosa CityBus,Santa Rosa Call Ahead Service",Metropolitan Transportation Commission,both,both,True,Name Match; NTD
11,City of Visalia,NaN,NaN,90091,City of Visalia,City of Visalia,Visalia Transit,VT,"City, County or Local Government Unit or Depar...",City/Town,NaN,Full Reporter,Visalia,361.0,654006.0,"V-LINE,Visalia Transit,Visalia Dial-A-Ride",Tulare County Association of Governments,both,both,True,Name Match; NTD
13,Colusa County Transit Agency,NaN,NaN,9R02-91112,Colusa County Transit Agency,Colusa County Transit Agency,NaN,CCTA,"City, County or Local Government Unit or Depar...",Independent Agency,NaN,Rural Reporter,Colusa,74.0,306002.0,Colusa County Transit,NaN,both,both,True,Name Match; NTD
17,Eastern Sierra Transit Authority,NaN,NaN,9R02-91062,Eastern Sierra Transit Authority,Eastern Sierra Transit Authority,NaN,ESTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Rural Reporter,Bishop,99.0,926002.0,Eastern Sierra Transit Authority Community Rou...,NaN,both,both,True,Name Match; NTD
18,El Dorado County Transit Authority,NaN,NaN,90229,El Dorado County Transit Authority,El Dorado County Transit Authority,NaN,EDCTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Reduced Reporter,Diamond Springs,101.0,309002.0,"El Dorado Transit,Sacramento/South Lake Tahoe ...",Sacramento Area Council of Governments,both,both,True,Name Match; NTD
22,Humboldt Transit Authority,NaN,NaN,9R02-91036,Humboldt Transit Authority,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Rural Reporter,Eureka,135.0,112005.0,"Humboldt Transit Authority Dial-A-Ride,Redwood...",NaN,both,both,True,Name Match; NTD
23,Humboldt Transit Authority,NaN,NaN,A0009,Humboldt Transit Authority,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Group Plan Sponsor,Eureka,135.0,112005.0,"Humboldt Transit Authority Dial-A-Ride,Redwood...",NaN,both,both,True,Name Match; NTD
24,Imperial County Transportation Commission,NaN,NaN,90226,Imperial County Transportation Commission,Imperial County Transportation Commission,NaN,ICTC,Independent Public Agency or Authority of Tran...,Independent Agency,NaN,Full Reporter,El Centro,138.0,1113001.0,Imperial Valley Transit,Southern California Association of Governments,both,both,True,Name Match; NTD


In [100]:
df>>filter(_.organization_type_y.isnull())

,organization_name,ntd_id_x,ntd_id_y,ntd_id,agency_name,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,_merge,in_5310,compare_names,merge_status
2,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
15,Desert Access and Mobility Inc,NaN,NaN,NaN,NaN,Desert Access and Mobility Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
25,Independent Living Partnership,NaN,NaN,NaN,NaN,Independent Living Partnership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,433.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
38,North and South of Market Adult Day Health,NaN,NaN,NaN,NaN,North and South of Market Adult Day Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,445.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
39,North Valley Services,NaN,NaN,NaN,NaN,North Valley Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446.0,NaN,NaN,NaN,right_only,both,True,Name Match; Airtable
62,Vivalon Inc,NaN,NaN,NaN,NaN,Vivalon Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golden Gate Transit Paratransit,NaN,right_only,both,True,Name Match; Airtable
73,Avenidas,NaN,NaN,NaN,NaN,Avenidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401.0,NaN,Avenidas Door-to-Door,NaN,right_only,left_only,False,"Manual Match, Airtable"
77,Community Bridges Liftline,NaN,NaN,NaN,NaN,Community Bridges Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.0,NaN,Lift Line,NaN,right_only,left_only,False,"Manual Match, Airtable"
78,Community Partners In Caring,NaN,NaN,NaN,NaN,Partners in Caring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only,left_only,False,"Manual Match, Airtable"
79,Dignity Health Connected Living,NaN,NaN,NaN,NaN,Dignity Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.0,NaN,"Dignity Health Connected Living,Shasta Connect",NaN,right_only,left_only,False,"Manual Match, Airtable"


In [117]:
df = df.drop(columns={'agency_name','ntd_id_x','ntd_id_y', '_merge', 'in_5310','compare_names'})

In [119]:
df.head()

,organization_name,ntd_id,name,doing_business_as,reporter_acronym,organization_type_x,organization_type_y,roles,reporter_type,city,itp_id,opm_id_drmt,mobility_services_operated,mpo_rtpa,merge_status
0,Angel View Inc,NaN,Angel View Inc,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,NaN,NaN,Angel View,NaN,Name Match; Airtable
1,ARC Imperial Valley,NaN,ARC Imperial Valley,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,398.0,NaN,Arc Imperial Valley Transportation,NaN,Name Match; Airtable
2,Area 1 Agency on Aging,NaN,Area 1 Agency on Aging,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.0,NaN,NaN,NaN,Name Match; Airtable
3,Big Valley 50 Plus,NaN,Big Valley 50 Plus,NaN,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,404.0,NaN,Big Valley 50 Plus,NaN,Name Match; Airtable
4,Care-A-Van Transit Systems Inc,NaN,Care-A-Van Transit Systems Inc,NaN,NaN,NaN,Company,NaN,NaN,NaN,407.0,NaN,Care-A-Van Transit,NaN,Name Match; Airtable


## Organization_types 

In [103]:
## will export and condesnse organziation type columns into one

In [25]:
print(f"Of the {len(df)} Organizations using 5310 funds, {len(df>>filter(_.organization_type_y.isnull()))} have no org type information")

Of the 150 Organizations using 5310 funds, 57 have no org type information


In [104]:
df.organization_type_y.value_counts()

Non-Profit Organization                        48
Independent Agency                             27
City/Town                                      10
Company                                         2
MPO/RTPA                                        2
Council of Governments                          1
Joint Powers Agency                             1
Private-Non-Profit Corporation                  1
Consolidated Transportation Services Agency     1
Name: organization_type_y, dtype: int64

#### Non-Profit

In [106]:
print(f"There are {len(df>>filter(_.organization_type_y=='Non-Profit Organization')>>select(_.organization_name))} Non-Profit Organizations")

There are 48 Non-Profit Organizations


In [28]:
df>>filter(_.organization_type_y=='Non-Profit Organization')>>select(_.organization_name)

,organization_name
0,Angel View Inc
1,ARC Imperial Valley
3,Big Valley 50 Plus
5,Casa Allegra Community Services
6,Catholic Charities of the Diocese of Stockton
14,Common Ground Senior Services
16,Desert ARC
19,ElderHelp of San Diego
20,Friends of Children with Special Needs
21,Humboldt Senior Resource Center


#### Transit Provider (with NTDs)

In [107]:
print(f"There are {len(df>>filter(_.ntd_id.notnull())>>select(_.organization_name))} NTD Reporter Transit Providers")

There are 36 NTD Reporter Transit Providers


In [30]:
df>>filter(_.ntd_id.notnull())>>select(_.organization_name)

,organization_name
8,City of Rio Vista
9,City of Roseville
10,City of Santa Rosa
11,City of Visalia
12,City of Visalia
13,Colusa County Transit Agency
17,Eastern Sierra Transit Authority
18,El Dorado County Transit Authority
22,Humboldt Transit Authority
23,Humboldt Transit Authority


#### Independent Agency

In [110]:
print(f"There are {len(df>>filter(_.organization_type_y=='Independent Agency')>>select(_.organization_name))} Independent Organizations")

There are 27 Independent Organizations


In [109]:
df>>filter(_.organization_type_y=='Independent Agency')>>select(_.organization_name)

,organization_name
13,Colusa County Transit Agency
17,Eastern Sierra Transit Authority
18,El Dorado County Transit Authority
22,Humboldt Transit Authority
23,Humboldt Transit Authority
24,Imperial County Transportation Commission
27,Lake Transit Authority
29,Marin County Transit District
30,Mendocino Transit Authority
34,Monterey-Salinas Transit


#### City or Town

In [112]:
print(f"There are {len(df>>filter(_.organization_type_y=='City/Town')>>select(_.organization_name))} Cities and Towns")

There are 10 Cities and Towns


In [34]:
df>>filter(_.organization_type_y=='City/Town')>>select(_.organization_name)

,organization_name
7,City of Fontana
8,City of Rio Vista
9,City of Roseville
10,City of Santa Rosa
11,City of Visalia
12,City of Visalia
58,Town of Truckee
71,San Francisco Municipal Transportation Agency
74,City of Lafayette: Lamorinda Sprit Van Program
75,City of Vista


### Still unknown

In [115]:
print(f"We do not have organization type information for {len(df>>filter(_.merge_status=='No Match')>>select(_.organization_name))} organizations")

We do not have organization type information for 41 organizations


In [116]:
df>>filter(_.merge_status=='No Match')>>select(_.organization_name)

,organization_name
109,Choice in Aging
110,Consolidated Tribal Health Project Inc
111,County of Sacramento Department of Child Famil...
112,County of Sonoma Human Services Department Adu...
113,County of Ventura Health Services Agency
114,Dignity Community Care
115,Foothill AIDS Project
116,Mental Health Client Action Network (MHCAN)
117,NCI Affiliates
118,OPARC


In [121]:
#df.to_csv('5310_org_info.csv')